In [1]:
from emergency import EmergencyObservationFunction, EmergencySUMOEnvironment, emergency_reward

In [2]:
import os
from trips_generator.config import *
from trips_generator.weibull_trips import main
import subprocess


scene_src_probabilities = {
    "High Traffic Scenerio": [0.25, 0.25, 0.25, 0.25],
    "Low Traffic Scenerio": [0.25, 0.25, 0.25, 0.25],
    "NS-Traffic Scenerio": [0.45, 0.05, 0.45, 0.05],
    "EW-Traffic Scenerio": [0.05, 0.45, 0.05, 0.45],
}

scene_number_cars = {
    "High Traffic Scenerio": 1000,
    "Low Traffic Scenerio": 150,
    "NS-Traffic Scenerio": 500,
    "EW-Traffic Scenerio": 500,
}


if not os.path.exists("routes_dqn"):
    os.makedirs("routes_dqn")
    counter = 0
    
    for scene in scene_src_probabilities:
        for i in range(75):
            main(
                src_nodes,
                dst_nodes,
                scene_src_probabilities[scene],
                turn_prob,
                emergency_prob,
                shape,
                scene_number_cars[scene],
                total_timesteps,
            )
            subprocess.run(
                [
                    "duarouter",
                    "-n",
                    "./network_details/intersection.net.xml",  # Input network file
                    "-t",
                    "trips.trips.xml",  # Input trips file
                    "-o",
                    f"routes_dqn/intersection_{counter}.rou.xml",  # Output routes file
                ]
            )
            counter += 1
        os.remove("trips.trips.xml")

In [3]:
route_files = [f"routes_dqn/intersection_{i}.rou.xml" for i in range(300)]

In [4]:
env = EmergencySUMOEnvironment(
    net_file="network_details/intersection.net.xml",
    route_files=route_files,
    out_csv_name=f'outputs/EmergencyTrain/DQN',
    min_green=5,
    yellow_time=5,
    delta_time=10,
    use_gui=False,
    num_seconds=5400,
    observation_class=EmergencyObservationFunction,
    reward_fn=emergency_reward
)



 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 0 ACT 0 BUF 0)                      


In [5]:
from stable_baselines3 import DQN
from stable_baselines3.common.logger import configure

out_path = "outputs/EmergencyTrain/"
new_logger = configure(out_path, ["stdout", "csv"])
model = DQN(
        env=env,
        policy="MlpPolicy",
        learning_rate=0.0001,
        train_freq=1,
        learning_starts=10000,
        target_update_interval=1000,
        exploration_fraction=0.3,
        exploration_initial_eps=1,
        tau=0.005,
        exploration_final_eps=0.01,
        verbose=1,
    )

model.set_logger(new_logger)
model.learn(540 * 450, log_interval=1)
model.save('agents/dqn_emergency')

Logging to outputs/EmergencyTrain/
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 14ms, vehicles TOT 500 ACT 0 BUF 0)                 tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 495 ACT 1 BUF 0)                  
 Retrying in 1 seconds
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 540      |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.993    |
| time/               |          |
|    episodes         | 1        |
|    fps              | 6        |
|    time_elapsed     | 77       |
|    total_timesteps  | 540      |
----------------------------------
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 23ms, vehicles TOT 150 ACT 0 BUF 0)                 tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 27ms, vehicles TOT 147 ACT 1 BUF 0)                 
 Retrying in 1 seconds
----------------------------------
| rollout/           

KeyboardInterrupt: 